# Fantasy Premier League (FPL) Team Selector

This notebook provides a comprehensive workflow for Fantasy Premier League (FPL) team selection. It combines:
1. Data extraction from the FPL API
2. Team optimization using linear programming
3. Data storage capabilities for historical analysis

## Features
- Fetch the latest FPL player, team, and position data
- Optimize team selection based on player performance and budget constraints
- Customize selection parameters (budget, team preferences, etc.)
- Store data to Azure SQL Database (optional)
- Display team selection with detailed breakdowns

## How to Use
1. Run the notebook from top to bottom
2. Review the selected team output
3. Adjust parameters in the "Customization" section if needed
4. (Optional) Configure database settings for data storage

*Last Updated: May 2025*

## Install Required Packages

Run this cell to install all necessary packages for the notebook:

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install requests pandas pulp pyodbc sqlalchemy tqdm python-dotenv

## Import Libraries

Import all the required libraries for data fetching, processing, and optimization:

In [ ]:
# Import required libraries
import requests
import json
import pandas as pd
import pulp as lp
import os
from tqdm.auto import tqdm
from datetime import datetime
from dotenv import load_dotenv
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Only import database libraries if needed for storage
try:
    import pyodbc
    from sqlalchemy import create_engine
    database_imports_successful = True
except ImportError:
    database_imports_successful = False
    print("Database libraries not available. Database storage features will be disabled.")

# Load environment variables from .env file (if it exists)
load_dotenv(verbose=False)

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## FPL Data Extraction

This section fetches the latest data from the Fantasy Premier League API:

In [ ]:
# Base FPL URL for all API endpoints
base_fpl_url = 'https://fantasy.premierleague.com/api/'

def fetch_fpl_data():
    """
    Fetch FPL data from the API and return as a dictionary
    """
    print("Fetching latest FPL data...")
    bootstrap_data = requests.get(base_fpl_url + 'bootstrap-static/').json()
    print("Data successfully retrieved!")
    return bootstrap_data

# Fetch the latest FPL data
fpl_data = fetch_fpl_data()

# Create DataFrames for players, teams, and positions
players_df = pd.json_normalize(fpl_data['elements'])
teams_df = pd.json_normalize(fpl_data['teams'])
positions_df = pd.json_normalize(fpl_data['element_types'])

# Display summary of fetched data
print(f"Fetched data for {len(players_df)} players, {len(teams_df)} teams, and {len(positions_df)} positions.")

## Player Data Preparation

Process player data to prepare it for the optimization model:

In [ ]:
def prepare_player_data(players_df, teams_df, positions_df):
    """
    Prepare player data for the optimization model
    """
    # Create a position name mapping
    position_names = positions_df.set_index('id')['singular_name'].to_dict()
    
    # Create a team name mapping
    team_names = teams_df.set_index('id')['name'].to_dict()
    
    # Create a dictionary to store player data
    players_data = {}
    
    # Process each player
    for _, player in players_df.iterrows():
        # Create player name
        player_name = f"{player['first_name']} {player['second_name']}"
        
        # Store player data
        players_data[player_name] = {
            "id": player['id'],
            "position": player['element_type'],
            "position_name": position_names[player['element_type']],
            "points_total": player['total_points'],
            "points_per_game": float(player['points_per_game']),
            "cost": player['now_cost'] / 10,  # Convert to millions
            "team": player['team'],
            "team_name": team_names[player['team']],
            "minutes": player['minutes'],
            "form": float(player['form']) if player['form'] else 0,
            "selected_by_percent": float(player['selected_by_percent']),
            "status": player['status'],
            "chance_of_playing_next_round": player['chance_of_playing_next_round']
        }
    
    print(f"Prepared data for {len(players_data)} players")
    return players_data, team_names

# Prepare player data
players_data, team_names = prepare_player_data(players_df, teams_df, positions_df)

## Customization Section

Customize your team selection parameters here:

In [ ]:
# ===== CUSTOMIZATION PARAMETERS =====
# Adjust these parameters based on your preferences

# Budget
total_budget = 100.0  # Total budget in millions

# Position constraints
max_starting_11_players_per_position = {
    1: 1,  # Goalkeeper
    2: 5,  # Defender
    3: 5,  # Midfielder
    4: 3,  # Forward
}

min_starting_11_players_per_position = {
    1: 1,  # Goalkeeper
    2: 3,  # Defender
    3: 2,  # Midfielder
    4: 1,  # Forward
}

max_squad_players_per_position = {
    1: 2,  # Goalkeeper
    2: 5,  # Defender
    3: 5,  # Midfielder
    4: 3,  # Forward
}

# Player filters
min_minutes_played = 90  # Minimum minutes played to consider a player
min_chance_of_playing = 75  # Minimum chance of playing (percentage)

# Scoring method: 'total_points', 'points_per_game', 'form', or 'custom'
scoring_method = 'points_per_game'

# Custom scoring weights (only used if scoring_method = 'custom')
custom_weights = {
    'total_points': 0.5,
    'form': 0.3,
    'points_per_game': 0.2
}

# Team preferences (0 = neutral, positive = prefer, negative = avoid)
team_preferences = {
    # Examples (update with your preferences):
    # 'Arsenal': 1,  # Prefer Arsenal players
    # 'Man City': 0.5,  # Slightly prefer Man City players
    # 'Norwich': -1,  # Avoid Norwich players
}

# Specific players to include or exclude
must_include_players = [
    # Example: "Erling Haaland"
]

must_exclude_players = [
    # Example: "Player Name"
]

# ===== END OF CUSTOMIZATION =====

# Apply filters to player data
filtered_players_data = {
    player: data for player, data in players_data.items()
    if (data['minutes'] >= min_minutes_played) and 
       (data['chance_of_playing_next_round'] is None or 
        data['chance_of_playing_next_round'] >= min_chance_of_playing) and
       (player not in must_exclude_players)
}

# Apply scoring method
if scoring_method == 'total_points':
    score_key = 'points_total'
elif scoring_method == 'points_per_game':
    score_key = 'points_per_game'
elif scoring_method == 'form':
    score_key = 'form'
elif scoring_method == 'custom':
    # Create custom score
    for player, data in filtered_players_data.items():
        custom_score = (
            data['total_points'] * custom_weights['total_points'] +
            data['form'] * custom_weights['form'] +
            data['points_per_game'] * custom_weights['points_per_game']
        )
        filtered_players_data[player]['custom_score'] = custom_score
    score_key = 'custom_score'

# Apply team preferences
for player, data in filtered_players_data.items():
    team = data['team_name']
    if team in team_preferences:
        # Adjust score based on team preference
        filtered_players_data[player]['adjusted_score'] = (
            data[score_key] * (1 + team_preferences[team])
        )
    else:
        filtered_players_data[player]['adjusted_score'] = data[score_key]

final_score_key = 'adjusted_score'

print(f"Using {len(filtered_players_data)} players after filtering")
print(f"Scoring method: {scoring_method}")

## Team Optimization Model

This section creates and solves the linear programming model to find the optimal team:

In [ ]:
def optimize_team(players_data, score_key='adjusted_score'):
    """
    Create and solve the optimization model
    """
    print("Setting up optimization model...")
    
    # Create a PuLP linear programming problem
    model = lp.LpProblem("FantasyFootballTeam", lp.LpMaximize)
    
    # Create binary variables for each player
    player_vars_starting_11 = {
        player: lp.LpVariable(f"starting_11_{player}", cat=lp.LpBinary) 
        for player in players_data.keys()
    }
    
    player_vars_squad = {
        player: lp.LpVariable(f"squad_{player}", cat=lp.LpBinary) 
        for player in players_data.keys()
    }
    
    # Objective function: maximize total score from the starting 11
    model += lp.lpSum(
        players_data[player][score_key] * player_vars_starting_11[player] 
        for player in players_data.keys()
    ), "TotalScore"
    
    # Budget constraint for the whole squad
    model += lp.lpSum(
        players_data[player]["cost"] * player_vars_squad[player] 
        for player in players_data.keys()
    ) <= total_budget, "Budget"
    
    # Squad size constraint
    model += lp.lpSum(
        player_vars_squad[player] for player in players_data.keys()
    ) == 15, "SquadSize"
    
    # Ensure the starting 11 is part of the squad
    for player in players_data.keys():
        model += player_vars_starting_11[player] <= player_vars_squad[player], f"Starting11InSquad_{player}"
    
    # Ensure exactly 11 players in the starting 11
    model += lp.lpSum(
        player_vars_starting_11[player] for player in players_data.keys()
    ) == 11, "Starting11Size"
    
    # Position constraints for the starting 11
    for position, min_count in min_starting_11_players_per_position.items():
        model += lp.lpSum(
            player_vars_starting_11[player] 
            for player in players_data.keys() 
            if players_data[player]["position"] == position
        ) >= min_count, f"MinStarting11PlayersPosition{position}"
    
    for position, max_count in max_starting_11_players_per_position.items():
        model += lp.lpSum(
            player_vars_starting_11[player] 
            for player in players_data.keys() 
            if players_data[player]["position"] == position
        ) <= max_count, f"MaxStarting11PlayersPosition{position}"
    
    # Maximum players per position for the squad
    for position, max_count in max_squad_players_per_position.items():
        model += lp.lpSum(
            player_vars_squad[player] 
            for player in players_data.keys() 
            if players_data[player]["position"] == position
        ) <= max_count, f"MaxSquadPlayersPosition{position}"
    
    # Constraint: no more than 3 players from any single team in the squad
    for team_id in set(data["team"] for data in players_data.values()):
        model += lp.lpSum(
            player_vars_squad[player] 
            for player in players_data.keys() 
            if players_data[player]["team"] == team_id
        ) <= 3, f"MaxPlayersPerTeam_{team_id}"
    
    # Add constraints for must-include players
    for player in must_include_players:
        if player in players_data:
            model += player_vars_squad[player] == 1, f"MustInclude_{player}"
    
    print("Solving optimization model...")
    # Solve the model
    solver = lp.PULP_CBC_CMD(msg=False)
    model.solve(solver)
    
    # Check if solution found
    if model.status != lp.LpStatusOptimal:
        print("No optimal solution found. Please check your constraints.")
        return None, None, None
    
    # Extract the selected players
    selected_players = [
        player for player in players_data.keys() 
        if lp.value(player_vars_squad[player]) > 0.5  # Using 0.5 as threshold for binary vars
    ]
    
    # Extract the starting 11
    starting_11 = [
        player for player in players_data.keys() 
        if lp.value(player_vars_starting_11[player]) > 0.5
    ]
    
    # Get bench players (in squad but not starting)
    bench_players = [player for player in selected_players if player not in starting_11]
    
    print(f"Optimization complete! Selected {len(selected_players)} players.")
    return selected_players, starting_11, bench_players

# Run the optimization
selected_players, starting_11, bench_players = optimize_team(filtered_players_data, final_score_key)

## Display Results

Show the optimized team selection:

In [ ]:
def display_team_selection(selected_players, starting_11, bench_players, players_data):
    """
    Display the selected team in a readable format
    """
    if not selected_players:
        print("No valid team found. Please adjust your constraints.")
        return
    
    # Calculate total cost and score
    total_cost = sum(players_data[player]["cost"] for player in selected_players)
    total_score = sum(players_data[player][final_score_key] for player in starting_11)
    
    # Determine captain (player with highest score)
    captain = max(starting_11, key=lambda x: players_data[x][final_score_key])
    vice_captain = sorted(
        [p for p in starting_11 if p != captain], 
        key=lambda x: players_data[x][final_score_key]
    )[-1]  # Second highest score
    
    # Group players by position
    position_map = {1: "GK", 2: "DEF", 3: "MID", 4: "FWD"}
    starting_11_by_position = {}
    bench_by_position = {}
    
    for pos_id in sorted(position_map.keys()):
        starting_11_by_position[pos_id] = [
            p for p in starting_11 if players_data[p]["position"] == pos_id
        ]
        bench_by_position[pos_id] = [
            p for p in bench_players if players_data[p]["position"] == pos_id
        ]
    
    # Print results
    print("\n" + "="*60)
    print(f"FANTASY PREMIER LEAGUE TEAM SELECTION ({datetime.now().strftime('%Y-%m-%d')})")
    print("="*60)
    
    print(f"\nTotal Budget: £{total_budget}m")
    print(f"Team Cost: £{total_cost:.1f}m")
    print(f"Remaining: £{total_budget - total_cost:.1f}m")
    print(f"Total Score: {total_score:.1f}")
    
    # Print formation
    formation = "-".join(str(len(starting_11_by_position[pos])) for pos in [2, 3, 4])
    print(f"\nFormation: {formation}")
    
    # Display Starting 11
    print("\n" + "="*60)
    print("STARTING XI")
    print("="*60)
    
    for pos_id in sorted(position_map.keys()):
        pos_name = position_map[pos_id]
        print(f"\n{pos_name}:")
        
        for player in starting_11_by_position[pos_id]:
            data = players_data[player]
            captain_mark = " (C)" if player == captain else " (VC)" if player == vice_captain else ""
            print(f"  {player}{captain_mark} | £{data['cost']}m | {data['team_name']} | Pts: {data[final_score_key]:.1f}")
    
    # Display Bench
    print("\n" + "="*60)
    print("BENCH")
    print("="*60)
    
    for pos_id in sorted(position_map.keys()):
        for player in bench_by_position[pos_id]:
            data = players_data[player]
            print(f"  {player} | {position_map[pos_id]} | £{data['cost']}m | {data['team_name']} | Pts: {data[final_score_key]:.1f}")
    
    # Team distribution
    print("\n" + "="*60)
    print("TEAM DISTRIBUTION")
    print("="*60)
    
    team_counts = {}
    for player in selected_players:
        team = players_data[player]["team_name"]
        team_counts[team] = team_counts.get(team, 0) + 1
    
    for team, count in sorted(team_counts.items(), key=lambda x: x[1], reverse=True):
        if count > 0:
            print(f"  {team}: {count} players")
    
    print("\n" + "="*60)

# Display the optimized team
if selected_players:
    display_team_selection(selected_players, starting_11, bench_players, filtered_players_data)

## Validation and Testing

Run these tests to ensure your team meets all FPL rules:

In [ ]:
def validate_team(selected_players, starting_11, bench_players, players_data):
    """
    Validate the selected team meets all FPL rules
    """
    if not selected_players:
        print("No team to validate.")
        return False
    
    print("\nRunning validation tests...")
    all_tests_passed = True
    
    # Test 1: Total squad size
    test_result = len(selected_players) == 15
    print(f"Test 1: Squad size is 15 - {'PASS' if test_result else 'FAIL'}")
    all_tests_passed = all_tests_passed and test_result
    
    # Test 2: Starting 11 size
    test_result = len(starting_11) == 11
    print(f"Test 2: Starting 11 size is 11 - {'PASS' if test_result else 'FAIL'}")
    all_tests_passed = all_tests_passed and test_result
    
    # Test 3: Budget constraint
    total_cost = sum(players_data[player]["cost"] for player in selected_players)
    test_result = total_cost <= total_budget
    print(f"Test 3: Total cost (£{total_cost:.1f}m) within budget (£{total_budget}m) - {'PASS' if test_result else 'FAIL'}")
    all_tests_passed = all_tests_passed and test_result
    
    # Test 4: Position constraints for squad
    for position, max_count in max_squad_players_per_position.items():
        position_count = sum(1 for player in selected_players if players_data[player]["position"] == position)
        test_result = position_count <= max_count
        print(f"Test 4.{position}: Squad has {position_count}/{max_count} {positions_df.loc[positions_df['id'] == position, 'plural_name'].values[0]} - {'PASS' if test_result else 'FAIL'}")
        all_tests_passed = all_tests_passed and test_result
    
    # Test 5: Position constraints for starting 11
    for position, min_count in min_starting_11_players_per_position.items():
        position_count = sum(1 for player in starting_11 if players_data[player]["position"] == position)
        test_result = position_count >= min_count
        print(f"Test 5.{position}: Starting 11 has at least {min_count} {positions_df.loc[positions_df['id'] == position, 'singular_name'].values[0]}(s) - {'PASS' if test_result else 'FAIL'}")
        all_tests_passed = all_tests_passed and test_result
    
    # Test 6: Maximum 3 players per team
    from collections import Counter
    team_counts = Counter(players_data[player]["team"] for player in selected_players)
    violations = [(team_id, count) for team_id, count in team_counts.items() if count > 3]
    test_result = len(violations) == 0
    print(f"Test 6: Maximum 3 players per team - {'PASS' if test_result else 'FAIL'}")
    if not test_result:
        for team_id, count in violations:
            team_name = teams_df.loc[teams_df['id'] == team_id, 'name'].values[0]
            print(f"  - {team_name} has {count} players (max is 3)")
    all_tests_passed = all_tests_passed and test_result
    
    # Overall result
    print(f"\nValidation {'PASSED' if all_tests_passed else 'FAILED'}")
    return all_tests_passed

# Validate the team
if selected_players:
    validate_team(selected_players, starting_11, bench_players, filtered_players_data)

## Optional: Save Data to Database

This section allows you to save your FPL data to an Azure SQL Database for historical analysis:

In [ ]:
def save_to_database():
    """
    Save data to Azure SQL Database (optional)
    """
    if not database_imports_successful:
        print("Database libraries not available. Skipping database storage.")
        return
    
    # Check if database environment variables are set
    server_name = os.getenv('server_name')
    database_name = os.getenv('database_name')
    username_db = os.getenv('username_db')
    password = os.getenv('password')
    
    if not all([server_name, database_name, username_db, password]):
        print("Database environment variables not set. Skipping database storage.")
        print("To enable database storage, create a .env file with:")
        print("  server_name=your_server_name")
        print("  database_name=your_database_name")
        print("  username_db=your_username")
        print("  password=your_password")
        return
    
    print("\nSaving data to database...")
    
    # Add timestamp to all dataframes
    current_time = datetime.now()
    players_df['timestamp'] = current_time
    teams_df['timestamp'] = current_time
    positions_df['timestamp'] = current_time
    
    try:
        # Establish connection to SQL Server
        connection_string = f"mssql+pyodbc://{username_db}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"
        engine = create_engine(connection_string)
        
        # Write data to SQL Server
        print("Writing players data...")
        players_df.to_sql(name='staging_players_all', con=engine, schema='dbo', if_exists='replace', index=False)
        
        print("Writing teams data...")
        teams_df.to_sql(name='staging_teams_all', con=engine, schema='dbo', if_exists='replace', index=False)
        
        print("Writing positions data...")
        # Fix sub_positions_locked column if needed
        if 'sub_positions_locked' in positions_df.columns:
            positions_df['sub_positions_locked'] = positions_df['sub_positions_locked'].apply(
                lambda x: int(x[0]) if isinstance(x, list) and len(x) > 0 else None
            )
        positions_df.to_sql(name='staging_positions', con=engine, schema='dbo', if_exists='replace', index=False)
        
        # Save selected team data
        if selected_players:
            # Create a DataFrame for selected players
            selected_team_data = []
            for player in selected_players:
                data = filtered_players_data[player]
                is_starting = player in starting_11
                is_captain = player == max(starting_11, key=lambda x: filtered_players_data[x][final_score_key])
                
                selected_team_data.append({
                    'player_name': player,
                    'player_id': data['id'],
                    'position_id': data['position'],
                    'position_name': data['position_name'],
                    'team_id': data['team'],
                    'team_name': data['team_name'],
                    'cost': data['cost'],
                    'points': data[final_score_key],
                    'is_starting': is_starting,
                    'is_captain': is_captain,
                    'selection_date': current_time
                })
            
            selected_team_df = pd.DataFrame(selected_team_data)
            print("Writing selected team data...")
            selected_team_df.to_sql(name='fpl_selected_teams', con=engine, schema='dbo', if_exists='append', index=False)
        
        print("Data successfully saved to database!")
        
    except Exception as e:
        print(f"Error saving to database: {str(e)}")

# Ask user if they want to save to database
save_choice = input("Do you want to save data to the database? (yes/no): ").lower()
if save_choice.startswith('y'):
    save_to_database()
else:
    print("Skipping database storage.")

## Team History (If Database Used)

This section allows you to view your team selection history if you're using the database:

In [ ]:
def view_team_history():
    """
    View history of selected teams from the database
    """
    if not database_imports_successful:
        print("Database libraries not available. Cannot view team history.")
        return
    
    # Check if database environment variables are set
    server_name = os.getenv('server_name')
    database_name = os.getenv('database_name')
    username_db = os.getenv('username_db')
    password = os.getenv('password')
    
    if not all([server_name, database_name, username_db, password]):
        print("Database environment variables not set. Cannot view team history.")
        return
    
    try:
        # Establish connection to SQL Server
        connection_string = f"mssql+pyodbc://{username_db}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"
        engine = create_engine(connection_string)
        
        # Query to get distinct selection dates
        query_dates = "SELECT DISTINCT FORMAT(selection_date, 'yyyy-MM-dd') as date FROM dbo.fpl_selected_teams ORDER BY date DESC"
        dates_df = pd.read_sql(query_dates, engine)
        
        if dates_df.empty:
            print("No team history found in the database.")
            return
        
        print("\nTeam selection history dates:")
        for i, date in enumerate(dates_df['date']):
            print(f"{i+1}. {date}")
        
        choice = input("\nEnter the number of the date to view (or 'q' to quit): ")
        if choice.lower() == 'q':
            return
        
        try:
            index = int(choice) - 1
            if 0 <= index < len(dates_df):
                selected_date = dates_df.iloc[index]['date']
                
                # Query to get team for selected date
                query_team = f"""
                SELECT player_name, position_name, team_name, cost, points, 
                       is_starting, is_captain
                FROM dbo.fpl_selected_teams
                WHERE FORMAT(selection_date, 'yyyy-MM-dd') = '{selected_date}'
                ORDER BY position_id, is_starting DESC, points DESC
                """
                team_df = pd.read_sql(query_team, engine)
                
                if team_df.empty:
                    print(f"No team data found for {selected_date}.")
                    return
                
                # Display the historical team
                print("\n" + "="*60)
                print(f"TEAM SELECTION FOR {selected_date}")
                print("="*60)
                
                # Split into starting and bench
                starting_df = team_df[team_df['is_starting']]
                bench_df = team_df[~team_df['is_starting']]
                
                # Display starting XI
                print("\nSTARTING XI:")
                for _, row in starting_df.iterrows():
                    captain_mark = " (C)" if row['is_captain'] else ""
                    print(f"  {row['player_name']}{captain_mark} | {row['position_name']} | £{row['cost']}m | {row['team_name']} | Pts: {row['points']:.1f}")
                
                # Display bench
                print("\nBENCH:")
                for _, row in bench_df.iterrows():
                    print(f"  {row['player_name']} | {row['position_name']} | £{row['cost']}m | {row['team_name']} | Pts: {row['points']:.1f}")
                
                # Show total cost
                total_cost = team_df['cost'].sum()
                print(f"\nTotal cost: £{total_cost:.1f}m")
                
            else:
                print("Invalid selection.")
        except ValueError:
            print("Invalid input. Please enter a number.")
            
    except Exception as e:
        print(f"Error accessing database: {str(e)}")

# Ask user if they want to view team history
history_choice = input("Do you want to view team selection history? (yes/no): ").lower()
if history_choice.startswith('y'):
    view_team_history()

## Player Statistics Lookup

Use this section to look up detailed statistics for specific players:

In [ ]:
def player_lookup():
    """
    Allow the user to look up player statistics
    """
    print("\n" + "="*60)
    print("PLAYER LOOKUP")
    print("="*60)
    
    search_term = input("\nEnter player name to search (or 'q' to quit): ")
    if search_term.lower() == 'q':
        return
    
    # Find matching players
    matching_players = [
        player for player in filtered_players_data.keys()
        if search_term.lower() in player.lower()
    ]
    
    if not matching_players:
        print(f"No players found matching '{search_term}'.")
        return
    
    # Display matches
    print(f"\nFound {len(matching_players)} matching players:")
    for i, player in enumerate(matching_players):
        data = filtered_players_data[player]
        print(f"{i+1}. {player} ({data['team_name']}) - £{data['cost']}m")
    
    choice = input("\nEnter the number of the player to view (or 'q' to quit): ")
    if choice.lower() == 'q':
        return
    
    try:
        index = int(choice) - 1
        if 0 <= index < len(matching_players):
            selected_player = matching_players[index]
            data = filtered_players_data[selected_player]
            
            print("\n" + "="*60)
            print(f"PLAYER DETAILS: {selected_player}")
            print("="*60)
            print(f"Team: {data['team_name']}")
            print(f"Position: {data['position_name']}")
            print(f"Cost: £{data['cost']}m")
            print(f"Total Points: {data['points_total']}")
            print(f"Points Per Game: {data['points_per_game']}")
            print(f"Form: {data['form']}")
            print(f"Minutes Played: {data['minutes']}")
            print(f"Selected by: {data['selected_by_percent']}% of managers")
            print(f"Status: {data['status']}")
            
            if data['chance_of_playing_next_round'] is not None:
                print(f"Chance of Playing Next Round: {data['chance_of_playing_next_round']}%")
            
            # Get detailed history if available
            if database_imports_successful:
                view_detailed_history = input("\nDo you want to view detailed history? (yes/no): ").lower()
                if view_detailed_history.startswith('y'):
                    try:
                        # Call function to fetch gameweek history from API
                        player_id = data['id']
                        player_history = get_player_gameweek_history(player_id)
                        
                        if not player_history.empty:
                            print("\nGameweek History:")
                            for _, row in player_history.iterrows():
                                print(f"GW{row['round']}: {row['total_points']} pts | {row['minutes']} mins | Goals: {row['goals_scored']} | Assists: {row['assists']}")
                        else:
                            print("No gameweek history available.")
                    except Exception as e:
                        print(f"Error fetching player history: {str(e)}")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Invalid input. Please enter a number.")

def get_player_gameweek_history(player_id):
    """
    Get gameweek history for a player from the FPL API
    """
    player_history_url = base_fpl_url + f'element-summary/{player_id}/'
    try:
        response = requests.get(player_history_url)
        data = response.json()
        history_df = pd.DataFrame(data['history'])
        return history_df
    except Exception:
        return pd.DataFrame()  # Return empty DataFrame on error

# Ask user if they want to look up player statistics
lookup_choice = input("Do you want to look up player statistics? (yes/no): ").lower()
if lookup_choice.startswith('y'):
    player_lookup()

## Conclusion

This notebook has provided a comprehensive workflow for FPL team selection, including:

1. Data extraction from the FPL API
2. Team optimization using linear programming
3. Detailed team analysis and validation
4. Optional data storage for historical analysis
5. Player statistics lookup

You can run this notebook each gameweek to get your optimal FPL team selection. Adjust the parameters in the "Customization" section to fine-tune your team based on your preferences and strategy.

Remember to check the official Fantasy Premier League website for deadlines, injuries, and other important information before finalizing your team.

Good luck!